In [388]:
import pandas as pd
import numpy as np
import math
import time
from pandas.api.types import CategoricalDtype
import gspread
import os

In [389]:
# key parameters

max_year = 2023
# min_year = 2021
# todo change when add other files
min_year = 2021

client_secret = "Desktop/GEM_INFO/client_secret.json"
client_secret_full_path = os.path.expanduser("~/") + client_secret

gogpt_dash_file = 'Global Oil and Gas Plant Tracker (GOGPT) compiled 2023-08-18.xlsx'
# todo add the other four files

gogpt_dash_file_status_1 = 'Global Gas Plant Tracker (GGPT) 2023-02.xlsx'
gogpt_dash_file_status_2 = 'Global Gas Plant Tracker 2022-08.xlsx'
gogpt_dash_file_status_3 = 'Global Gas Plant Tracker (GGPT) 2022-02.xlsx'
gogpt_dash_file_status_4 = 'Global Gas Plant Tracker 2021-10.xlsx'

gogpt_dash_path = os.path.expanduser('/Users/gem-tah/Desktop/GEM_INFO/GEM_WORK/GOGPT/GOGPT-Dashboard/data/')
print(gogpt_dash_path)
gogpt_dash_path_status = os.path.expanduser('/Users/gem-tah/Desktop/GEM_INFO/GEM_WORK/GOGPT/GOGPT-Dashboard/data/pre-2023-08/')


/Users/gem-tah/Desktop/GEM_INFO/GEM_WORK/GOGPT/GOGPT-Dashboard/data/


In [390]:
gogpt_dash_xl = pd.ExcelFile(gogpt_dash_path + gogpt_dash_file)
# todo add the other four files
gogpt_dash_xl_status_1 = pd.ExcelFile(gogpt_dash_path_status + gogpt_dash_file_status_1)
gogpt_dash_xl_status_2 = pd.ExcelFile(gogpt_dash_path_status + gogpt_dash_file_status_2)
gogpt_dash_xl_status_3 = pd.ExcelFile(gogpt_dash_path_status + gogpt_dash_file_status_3)
gogpt_dash_xl_status_4 = pd.ExcelFile(gogpt_dash_path_status + gogpt_dash_file_status_4)


In [391]:
def gspread_access_file_read_only(key, title):
    """
    key = Google Sheets unique key in the URL
    title = name of the sheet you want to read
    """
    gspread_creds = gspread.oauth(
        scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"],
        credentials_filename=client_secret_full_path,
        # authorized_user_filename=json_token_name,
    )
    gsheets = gspread_creds.open_by_key(key)
    # Access a specific tab
    spreadsheet = gsheets.worksheet(title)
    # expected_header option provided following: https://github.com/burnash/gspread/issues/1007
    df = pd.DataFrame(spreadsheet.get_all_records(expected_headers=[]))
    
    return df

def determine_year_month_from_file(gogpt_dash_file):
    yyyy_mm = gogpt_dash_file.split('.xlsx')[0].split(' ')[-1]
    month_int = int(yyyy_mm.split('-')[1])
    year_int = int(yyyy_mm.split('-')[0])

    print(f"year_int: {year_int} month_int: {month_int}")

    return month_int, year_int

In [392]:
# create list of countries to choose from (GEM country names)
# data in gogpt_status is most complete; example of Albania, which only has cancelled units, nothing else
# Todo make a function to reduce repetition here 
file_month, file_year = determine_year_month_from_file(gogpt_dash_file)

file_month_stat1, file_year_stat1 = determine_year_month_from_file(gogpt_dash_file_status_1)
file_month_stat2, file_year_stat2 = determine_year_month_from_file(gogpt_dash_file_status_2)
file_month_stat3, file_year_stat3 = determine_year_month_from_file(gogpt_dash_file_status_3)
file_month_stat4, file_year_stat4 = determine_year_month_from_file(gogpt_dash_file_status_4)

gogpt_orig = pd.read_excel(gogpt_dash_xl, sheet_name = 'Gas &amp; Oil Units')
gogpt_orig['Version month'] = file_month
gogpt_orig['Version year'] = file_year

gogpt_orig_status_1 = pd.read_excel(gogpt_dash_xl_status_1, sheet_name = 'Gas Units')
gogpt_orig_status_1['Version month'] = file_month_stat1
gogpt_orig_status_1['Version year'] = file_year_stat1

gogpt_orig_status_2 = pd.read_excel(gogpt_dash_xl_status_2, sheet_name = 'Gas plants - data')
gogpt_orig_status_2['Version month'] = file_month_stat2
gogpt_orig_status_2['Version year'] = file_year_stat2

gogpt_orig_status_3 = pd.read_excel(gogpt_dash_xl_status_3, sheet_name = 'Gas Units')
gogpt_orig_status_3['Version month'] = file_month_stat3
gogpt_orig_status_3['Version year'] = file_year_stat3

gogpt_orig_status_4 = pd.read_excel(gogpt_dash_xl_status_4, sheet_name = 'GGPT - Gas Units')
gogpt_orig_status_4['Version month'] = file_month_stat4
gogpt_orig_status_4['Version year'] = file_year_stat4



gogpt_country_list = gogpt_orig['Country'].sort_values().unique().tolist()
gogpt_country_list = ['all'] + gogpt_country_list

year_int: 2023 month_int: 8
year_int: 2023 month_int: 2
year_int: 2022 month_int: 8
year_int: 2022 month_int: 2
year_int: 2021 month_int: 10


In [393]:
# drop rows where capacity or country is na or not found
print(gogpt_orig.shape)
gogpt_orig = gogpt_orig.dropna(subset= ['Capacity (MW)', 'Country'])
gogpt_orig = gogpt_orig.drop(gogpt_orig[gogpt_orig['Capacity (MW)'] == 'not found'].index)
print(gogpt_orig.shape)

list_of_dfs = [gogpt_orig_status_1, gogpt_orig_status_2, gogpt_orig_status_3, gogpt_orig_status_4]

def clean_status_dfs(df_list):
    list_of_dfs_clean = []
    for df in list_of_dfs:
        print(df.shape)
        df = df.rename(columns={'Capacity elec. (MW)': 'Capacity (MW)'})
        df = df.dropna(subset=['Capacity (MW)', 'Country'])
        df = df.drop(df[df['Capacity (MW)'] == 'not found'].index)
        print(df.shape)
        list_of_dfs_clean.append(df)
    return list_of_dfs_clean

gogpt_orig_status_1, gogpt_orig_status_2, gogpt_orig_status_3, gogpt_orig_status_4 = clean_status_dfs(list_of_dfs)


(11787, 38)
(11774, 38)
(9898, 39)
(9886, 39)
(9578, 37)
(9554, 37)
(9350, 33)
(9323, 33)
(8957, 36)
(8937, 36)


In [394]:
print(gogpt_orig_status_4.shape)

(8937, 36)


### Map (upper left)
- pivot capacity by country in gogpt_orig to get map tab for gogpt input


In [395]:
# create pivot on capacities from country
# gogpt_orig is df, so pivot sum on column ['Capacity MW'] for all ['Country'] 
# remove unnecessary columns
# then groupby country and sum

# replace not found with 1
gogpt_map = gogpt_orig[['Country', 'Capacity (MW)']].copy() # .fillna(1).replace('not found', 1)

gogpt_map['Capacity (MW)'] = pd.to_numeric(gogpt_map['Capacity (MW)'])

gogpt_map = gogpt_map.groupby(['Country']).sum().reset_index()
# gogpt_map = gogpt_map.groupby(['Country'])[['Capacity (MW)']].sum().reset_index()

# gogpt_map = gogpt_map.groupby(by="Country")["Capacity (MW)"].sum()
# Convert "to int"
gogpt_map = gogpt_map.astype({'Capacity (MW)':'int'})



In [396]:
def create_country_code_df():

    gem_naming_convention_key = '1mtlwSJfWy1gbIwXVgpP3d6CcUEWo2OM0IvPD6yztGXI'

    # gspread way
    country_codes = gspread_access_file_read_only(gem_naming_convention_key, 'Countries')

    # clean up the codes to remove non-printing characters from wikipedia
    for col in country_codes.columns:
        if country_codes[col].dtype == object:
            country_codes[col] = country_codes[col].str.replace('\xa0', '', regex=False)

    # get rid of parenthetical footnotes at end of names & whitespace
    country_codes['ISO 3166 Country Name'] = country_codes['ISO 3166 Country Name'].str.split('[').str[0].str.strip()

    return country_codes

In [397]:
def create_country_name_conversions_dict():
    """
    Creates a dictionary for converting GEM standard country names to ISO 3166 versions.
    
    This downloads the GEM standard names file from Google Sheets using pygsheets,
    then pares down the df to only those in which GEM uses a different name than ISO 3166.

    Then it creates a dictionary, which can be used for converting from GEM to ISO.
    
    This is needed because Plotly's choropleth function uses ISO names for getting the outline for each country.
    """
    
    standard_country_names_key = '1mtlwSJfWy1gbIwXVgpP3d6CcUEWo2OM0IvPD6yztGXI'

    # gspread way
    df = gspread_access_file_read_only(standard_country_names_key, 'Countries')
    # keep only those with a mismatch
    name_diffs = df.copy()
    name_diffs = name_diffs[name_diffs['GEM name same as ISO 3166?']=='FALSE']
    # exclude those not in ISO
    name_diffs = name_diffs[name_diffs['ISO 3166 Country Name']!='NOT LISTED']
    name_diffs_dict = name_diffs.set_index('GEM Standard Country Name')['ISO 3166 Country Name'].to_dict()   
    
    return name_diffs_dict

In [398]:
# get map df
# gogpt_map = pd.read_excel(gogpt_dash_xl, sheet_name = 'Map')
# gogpt_map = gogpt_map.rename(columns={'Total operating Capacity': 'Capacity (MW)'})

# add any missing countries, to make sure that all countries in gogpt_country_list are in gogpt_map
missing_countries = [x for x in gogpt_country_list if x not in gogpt_map['Country'].tolist()]
if len(missing_countries) > 0:
    print(f"These countries were missing: {missing_countries}")
    missing_df = pd.DataFrame.from_dict({
            'Country': missing_countries, 
            'Capacity (MW)': [float(0)]*len(missing_countries)
        }, orient='columns')
    gogpt_map = pd.concat([gogpt_map, missing_df], sort=False)

# change GEM country names to ISO 3166
name_diffs_dict = create_country_name_conversions_dict()
gogpt_map['ISO 3166 Country Name'] = gogpt_map['Country'].replace(name_diffs_dict)

# Note: Kosovo isn't recognized in ISO 3166, so can't be shown on Plotly map on its own. 
# We could combine it with Serbia for display--but then our data in the dashboard wouldn't be the same as in our spreadsheets & other maps.
# Wikipedia said Kosovo declared independence from Serbia in 2008; it is only partially recognized.

# show countries in gogpt_map not in gogpt_country_list:
extraneous_countries = [x for x in gogpt_map['Country'].tolist() if x not in gogpt_country_list]
if len(extraneous_countries) > 0:
    print(f"Extraneous countries to be removed: {extraneous_countries}")

# keep only countries that are in gogpt_country_list
gogpt_map = gogpt_map[gogpt_map['Country'].isin(gogpt_country_list)]

# merge in ISO country codes (needed by Plotly)
country_codes = create_country_code_df()
gogpt_map = pd.merge(
    country_codes[['ISO 3166 Country Name', 'Country ISO 3166-1 alpha-3']],
    gogpt_map,
    on='ISO 3166 Country Name', 
    how='outer'
)
gogpt_map = gogpt_map.rename(columns={'Country ISO 3166-1 alpha-3': 'iso_alpha'})

# exclude those with no value for iso_alpha
# This excludes notes within the ISO dataset, e.g., "Akrotiri and Dhekelia – See United Kingdom, The."
# Unfortunately, this also excludes Kosovo from gogpt
gogpt_map = gogpt_map[gogpt_map['iso_alpha'].isna()==False]

These countries were missing: ['all']


In [399]:
# if capacity is 0, instead use 1, to avoid zero capacity leading to -inf log value
# similar approach is used in numpy log1p
# https://numpy.org/doc/stable/reference/generated/numpy.log1p.html
gogpt_map['capacity log10 + 1'] = np.log10(gogpt_map['Capacity (MW)'].replace(float(0), float(1)))

# create hover text
# (note: variable hover_text below is a Pandas Series)
hover_text = gogpt_map['Country'] + ': '
hover_text = hover_text + gogpt_map['Capacity (MW)'].map('{:,.0f}'.format) + ' MW'
# hide extra bit, e.g. 'trace 0'; based on https://plotly.com/python/reference/#scatter-hovertemplate
hover_text = hover_text + '<extra></extra>'
gogpt_map['hover_text'] = hover_text

### Status map
- from orig get status column, and country column, and capacity column groupby country and status, removing retired and cancelled
- do this for 2020, 2021, 2022, 2023 datasets and create column for which year it is then combine all dfs

In [400]:
list_of_dfs = gogpt_orig, gogpt_orig_status_1, gogpt_orig_status_2, gogpt_orig_status_3, gogpt_orig_status_4
accepted_statuses = ['announced', 'construction', 'mothballed', 'operating', 'pre-construction', 'shelved']


def clean_status_dfs_further(list_of_dfs):
    cleaned_status_dfs = []
    for df in list_of_dfs:
        df_uncondensed = df[['Country', 'Capacity (MW)', 'Status', 'Version month', 'Version year', 'GEM unit ID']].copy() 
        df_uncondensed['Status'] = df_uncondensed['Status'].replace({'proposed': 'pre-construction'})
        df_uncondensed = df_uncondensed[df_uncondensed['Status'].isin(accepted_statuses)]
        version_month = df_uncondensed['Version month'].values[0]
        version_year = df_uncondensed['Version year'].values[0]
        df_uncondensed = df_uncondensed[['Country', 'Capacity (MW)', 'Status']].copy()
        df_uncondensed = df_uncondensed.groupby(['Country','Status', 'Capacity (MW)']).sum(['Capacity (MW)']).reset_index()
        # df_uncondensed = df_uncondensed.rename(columns={
        # 'Status': version_year, 
        # })
        if version_month >= 6:
            xaxis_version_year = version_year + .5
        else:
            xaxis_version_year = version_year
        status_column = f'{xaxis_version_year}'
        df_uncondensed = df_uncondensed.rename(columns={
            'Status': status_column, 
            })
        # df_uncondensed['xaxis'] = xaxis_version_year
        cleaned_status_dfs.append(df_uncondensed)

    return cleaned_status_dfs

gogpt_status_uncondensed, gogpt_orig_status_1, gogpt_orig_status_2, gogpt_orig_status_3, gogpt_orig_status_4 = clean_status_dfs_further(list_of_dfs)

In [401]:
gogpt_status_uncondensed

,Country,2023.5,Capacity (MW)
0,Afghanistan,announced,59.0
1,Albania,announced,170.0
2,Albania,mothballed,52.2
3,Albania,mothballed,58.0
4,Albania,pre-construction,97.0
...,...,...,...
4527,Yemen,operating,132.0
4528,Yemen,shelved,60.0
4529,Zimbabwe,announced,150.0
4530,Zimbabwe,announced,350.0


In [402]:
gogpt_orig_status_1

,Country,2023,Capacity (MW)
0,Albania,pre-construction,97.0
1,Algeria,construction,631.0
2,Algeria,construction,633.0
3,Algeria,construction,725.0
4,Algeria,construction,1294.0
...,...,...,...
3734,Yemen,pre-construction,50.0
3735,Yemen,shelved,60.0
3736,Zimbabwe,announced,150.0
3737,Zimbabwe,announced,350.0


In [403]:
# gogpt_status_uncondensed = gogpt_orig[['Country', 'Capacity (MW)', 'Status', 'Version month', 'Version year']].copy() 
# # todo add the other four files
# accepted_statuses = ['announced', 'construction', 'mothballed', 'operating', 'pre-construction', 'shelved']
# # change proposed to pre-construction for status
# gogpt_status_uncondensed['Status'] = gogpt_status_uncondensed['Status'].replace({'proposed': 'pre-construction'})
# # filter out cancelled and retired and not found
# gogpt_status_uncondensed = gogpt_status_uncondensed[gogpt_status_uncondensed['Status'].isin(accepted_statuses)]


In [404]:
# version_month = gogpt_status_uncondensed_merged['Version month'].values[0]

# version_year = gogpt_status_uncondensed['Version year'].values[0]
# # if version_month >= 6:
# #     version_year = version_year + .5
# # else:
# #     pass
# print(version_year)
# gogpt_status_uncondensed = gogpt_status_uncondensed[['Country', 'Capacity (MW)', 'Status']].copy()

In [405]:
# gogpt_status_uncondensed['Capacity (MW)'] = gogpt_status_uncondensed['Capacity (MW)'].fillna(1).replace('not found', 1)

# gogpt_status_uncondensed = gogpt_status_uncondensed.groupby(['Country','Status']).sum().reset_index()
# gogpt_status_uncondensed = gogpt_status_uncondensed.rename(columns={
#         'Status': version_year, 
#     })

In [406]:
#merge all status dfs

gogpt_status_uncondensed_merged = pd.merge(
    gogpt_status_uncondensed,
    gogpt_orig_status_1,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

gogpt_status_uncondensed_merged = pd.merge(
    gogpt_status_uncondensed_merged,
    gogpt_orig_status_2,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

gogpt_status_uncondensed_merged = pd.merge(
    gogpt_status_uncondensed_merged,
    gogpt_orig_status_3,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

gogpt_status_uncondensed_merged = pd.merge(
    gogpt_status_uncondensed_merged,
    gogpt_orig_status_4,
    on=['Country', 'Capacity (MW)'],
    how='outer'
)

In [407]:
gogpt_status_uncondensed_merged

,Country,2023.5,Capacity (MW),2023,2022.5,2022,2021.5
0,Afghanistan,announced,59.0,NaN,NaN,NaN,NaN
1,Albania,announced,170.0,NaN,NaN,NaN,NaN
2,Albania,mothballed,52.2,NaN,NaN,NaN,NaN
3,Albania,mothballed,58.0,NaN,NaN,NaN,NaN
4,Albania,pre-construction,97.0,pre-construction,pre-construction,pre-construction,mothballed
...,...,...,...,...,...,...,...
30381,United States,NaN,1022.9,NaN,NaN,NaN,pre-construction
30382,United States,NaN,1171.4,NaN,NaN,NaN,pre-construction
30383,United States,NaN,1183.2,NaN,NaN,NaN,pre-construction
30384,Uzbekistan,NaN,53.0,NaN,NaN,NaN,operating


In [408]:
gogpt_status_uncondensed
# rename Status to be year
# groupby country and status 
# remove statuses we don't want 

,Country,2023.5,Capacity (MW)
0,Afghanistan,announced,59.0
1,Albania,announced,170.0
2,Albania,mothballed,52.2
3,Albania,mothballed,58.0
4,Albania,pre-construction,97.0
...,...,...,...
4527,Yemen,operating,132.0
4528,Yemen,shelved,60.0
4529,Zimbabwe,announced,150.0
4530,Zimbabwe,announced,350.0


In [409]:
def status_clean_data(df):
    for col in df.columns:
        if col not in ['Country', 'Capacity (MW)', 'xaxis']:
            # make the text lowercase
            print(f"Cleaning col {col}")
            df[col] = df[col].str.lower()

    return df

In [410]:
def sort_status(df):
    """
    convert column 'Status' to categorical
    https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html
    """
    
    status_order = [
        'operating',
        'mothballed',
        'announced',
        'pre-construction',
        'construction',
        # 'retired',
        'shelved',
        # 'cancelled',
    ]
    df['Status'] = df['Status'].astype(
        CategoricalDtype(status_order, ordered=False)
    )    
    df = df.sort_values(by=['Country', 'Status', 'Year'])
    
    return df

In [411]:
def status_test_capacity(df, df_init):
    """Runs at end of status_condense, to check that there was no change in the data.
    (Comparing version after condensing the data as it was before condensing the data.)
    
    There are entries with status NaN in the version of the data before condensing,
    because the table prior to that step has a row for each unit, and columns for each year.
    (Other distinguishing features for each unit aren't included in this table.)
    
    If there is no data for a unit for some of the years covered, then we have status NaN for those years.
    (Example: It was proposed only in 2018, then the years 2014-2017 have status NaN.)
    """

    df = df.reset_index()
        
    all_statuses = []
    # df.astype({'Year': 'int32'}).dtypes
    # reassign years with .5 so can do min on integer
    
    # df['Year'] = df['Year'].replace({2021.5: 2021, 2022.5: 2022, 2023.5:2023})
    
    # print(df['Year'])
    # df['Year'] = df['Year'].astype(int)
    # min_year = df['Year'].min()
    # print(min_year)


    for year in ['2021.5', '2022', '2022.5', '2023', '2023.5']:
        all_statuses += df_init[year].tolist() 

    all_statuses = list(set(all_statuses))

    for sel_status in all_statuses:
        print(f"Testing capacities for {sel_status}")
        for sel_year in ['2021.5', '2022', '2022.5', '2023', '2023.5']:
            sel_init = df_init[df_init[sel_year]==sel_status]
            sel_init_sum = sel_init['Capacity (MW)'].sum()

            # handle two different formats for df
            if 'Status' in df.columns:
                sel = df[(df['Year']==sel_year) & (df['Status']==sel_status)]
                sel_sum = sel['Capacity (MW)'].sum()
                
            else:
                sel = df[df[sel_year]==sel_status]
                sel_sum = sel['Capacity (MW)'].sum()

            # compare values
            abs_diff = abs(sel_sum - sel_init_sum)
            if abs_diff <= 1e-7:
                pass

            elif abs_diff > 1e-7:
                print("Error!" + f" Capacity difference for {sel_year} & {sel_status}.")
                print(f"Initial value: {sel_init_sum}")
                print(f"Current value: {sel_sum}")
            else:
                print("Unexpected case")
    
    # no return

In [412]:
def status_fill_in_missing(df):
    """
    For each country, for each status, fill in zero value if it is missing.
    """
    # year_range = range(df['Year'].min(), df['Year'].max() + 1)
    year_range = ['2021.5', '2022', '2022.5', '2023', '2023.5']
    statuses = df['Status'].dropna().unique().tolist()
    for country in df['Country'].unique().tolist():
        for status in statuses:
            for year in year_range:
                df_sel = df[(df['Year']==year) & (df['Country']==country) & (df['Status']==status)]
                if len(df_sel) == 0:
                    # there is a missing value; fill it in
                    # print(f"To fill in missing value for {country}, {year}, {status}") # for db
                    # values in dict below must be lists, to avoid error:
                    # "ValueError: If using all scalar values, you must pass an index"
                    new_df = pd.DataFrame.from_dict({
                                'Country': [country],
                                'Year': [year],
                                'Status': [status],
                                'Capacity (MW)': [float(0)],
                            }, orient='columns')
                    df = pd.concat([df, new_df], sort=False)

    df = df.sort_values(by=['Country', 'Year', 'Status'])
                    
    return df

In [413]:
def status_calculate_global_totals(df):    
    gogpt_status_all = df.groupby(['Year', 'Status'])[['Capacity (MW)']].sum().reset_index()
    gogpt_status_all.insert(0, 'Country', 'all')
    gogpt_status_all = sort_status(gogpt_status_all)
    
    df = pd.concat([gogpt_status_all, df], sort=False)

    return df

In [414]:
def status_condense(df_arg):
    """
    There are sometimes duplicates, for example if two units in a plant have the exact same capacity and history.
    So before setting the index to Country & Capacity, need to get rid of duplicates.
    """
    df = df_arg.copy()
    df_init = df_arg.copy()  # for test
    
    df = df.dropna(how='all')
    
    df = df.set_index(['Country', 'Capacity (MW)'])
    df = df.stack().reset_index()
        
    df = df.rename(columns={'level_2': 'Year', 0: 'Status'})
    
    df = df.groupby(['Country', 'Year', 'Status'])[['Capacity (MW)']].sum().reset_index()
    df = sort_status(df)

    # print(df.columns) # for db
    status_test_capacity(df, df_init)
    
    return df

In [415]:
def test_statuses(gogpt_status_uncondensed_merged):
    accepted_statuses = ['announced', 'construction', 'mothballed', 'operating', 'pre-construction', 'shelved']
    # TODO uncomment when have all the years
    for col in ['2021.5', '2022', '2022.5', '2023', '2023.5']:
    # print(version_year)
    # for col in [version_year]:
        ser = gogpt_status_uncondensed_merged[col].dropna()
        unaccepted = ser[~ser.isin(accepted_statuses)]
        if len(unaccepted) > 0:
            print(f"Found unaccepted statuses; len(ser): {len(ser)}")
            print(unaccepted.value_counts())
    # no return

In [416]:
# uncomment after getting the combined df with all years for status, country and capacity
# gogpt_status_uncondensed = pd.read_excel(gogpt_dash_xl, sheet_name = f'2014-{max_year}')
# gogpt_status_uncondensed = gogpt_orig[['Country', 'Capacity (MW)', 'Status']].copy()

gogpt_status_uncondensed_merged = status_clean_data(gogpt_status_uncondensed_merged)
test_statuses(gogpt_status_uncondensed_merged)

gogpt_status = status_condense(gogpt_status_uncondensed_merged)
gogpt_status = status_fill_in_missing(gogpt_status)
gogpt_status = status_calculate_global_totals(gogpt_status)

Cleaning col 2023.5
Cleaning col 2023
Cleaning col 2022.5
Cleaning col 2022
Cleaning col 2021.5
Testing capacities for mothballed
Testing capacities for shelved
Testing capacities for announced
Testing capacities for construction
Testing capacities for pre-construction
Testing capacities for operating
Testing capacities for nan


### Age & Technology (lower left)
- For bar chart of capacity by age & type
- From the sheet 'Plant type and age'
- same as coal, first make the age variable


In [417]:
# create age variable
gogpt_age = gogpt_orig[['Country', 'Capacity (MW)', 'Start year', 'Technology']].copy()
# gogpt_age = gogpt_age[['Country', 'Capacity (MW)', 'Start year', 'Technology']].fillna(0).replace('not found', 1)
# gogpt_age['Start year'] = gogpt_age['Start year'].fillna(0).replace('not found', 0)

gogpt_age = gogpt_age.dropna(subset= ['Start year', 'Technology'])

gogpt_age = gogpt_age.drop(gogpt_age[gogpt_age['Start year'] == 'not found'].index) 
# gogpt_age = gogpt_age.drop(gogpt_age[gogpt_age['Technology'] == 'not found'].index)

plant_ages = []
for value in gogpt_age['Start year']:
    if '-' in str(value):
        first_yr = int(value.split('-')[0].strip())
        sec_yr = int(value.split('-')[1].strip())
        if sec_yr < first_yr:
            print(f"whooops {sec_yr} is less than {first_yr}")
            plant_age = 'not valid'
            plant_ages.append(plant_age)
        elif first_yr >= max_year:
            plant_age = 0
            plant_ages.append(plant_age)
        else:
            plant_age = max_year - int(first_yr)
            plant_ages.append(plant_age)
    elif value == 'not found' or value == 0:
        # made up -1000
        # plant_age = 'not found'
        plant_age = 0
        plant_ages.append(plant_age)

    elif value >= max_year:
        plant_age = 0
        plant_ages.append(plant_age)
    else:
       plant_age = max_year - int(value)
       plant_ages.append(plant_age)

print(len(plant_ages))
print(gogpt_age.shape)

if len(plant_ages) != gogpt_age.shape[0]:
    print("Something went wrong with logic, missing a plant age for a row.")
else:
    gogpt_age['Plant Age'] = plant_ages
        
        

gogpt_age['Capacity (MW)'] = pd.to_numeric(gogpt_age['Capacity (MW)'])

# replace negative with 1
# gogpt_age['Plant Age'] = gogpt_age['Plant Age'] < 0 = 0

# gogpt_age = gogpt_age.groupby(['Country',  'Start year', 'Plant Age', 'Technology']).sum().reset_index()
# print(gogpt_age['Plant Age'].min()) # -18

# gogpt_age['Plant Age'] = gogpt_age['Plant Age'].apply(lambda x: 1 if x > 0 else 0)

gogpt_age.head()

10386
(10386, 4)


,Country,Capacity (MW),Start year,Technology,Plant Age
0,Algeria,338.0,2019,CC,4
1,Algeria,338.0,2019,CC,4
2,Algeria,338.0,2019,CC,4
3,Algeria,126.0,2010,GT,13
4,Algeria,126.0,2010,GT,13


In [418]:
def age_read_and_clean(gogpt_age):
    # rename technology to be human friendly 
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'GT', 'Gas Turbine'
    )
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'ST', 'Steam Turbine'
    )
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'CC', 'Combined Cycle'
    )
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'ICCC', 'Internal Combustion Combined Cycle'
    )   
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'ISCC', 'Integrated Solar Combined Cycle'
    ) 
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'IC', 'Internal Combustion'
    ) 
    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'AFC', 'Allam-Fetvedt Cycle'
    ) 
    #     gogpt_age['Technology'] = gogpt_age['Technology'].replace(
    #     'UISCC', 'Ultra-Integrated Solar Combined Cycle'
    # ) 

    # change not found technology to unknown

    gogpt_age['Technology'] = gogpt_age['Technology'].replace(
        'not found', 'Unknown'
    ) 

    return gogpt_age

def age_condense_data(df):
    print(len(df))

    # bin by decade
    for row in df.index:
        age = df.at[row, 'Plant Age']
        if age < 10:
            df.at[row, 'Decade'] = '0-9 years'
        elif age >= 10 and age < 20:
            df.at[row, 'Decade'] = '10-19 years'
        elif age >= 20 and age < 30:
            df.at[row, 'Decade'] = '20-29 years'
        elif age >= 30 and age < 40:
            df.at[row, 'Decade'] = '30-39 years'
        elif age >= 40 and age < 50:
            df.at[row, 'Decade'] = '40-49 years'
        elif age >= 50:
            df.at[row, 'Decade'] = '50+ years'
        else:
            print("Error!" + f" Issue with age for row {row}: {age}")

    df = df.groupby(['Country', 'Decade', 'Technology'])[['Capacity (MW)']].sum()
    df = df.reset_index()
    print(f'this is df after groupby: {df}')
    
    # unstack, then fill in zeros
    df = df.set_index(['Country', 'Decade', 'Technology']).unstack()
    df = df.droplevel(0, axis=1)
    
    for col in df.columns:
        df[col] = df[col].fillna(0)

    df = df.reset_index()
    df.columns.name = ''
    
    print(len(df))
    
    return df

def age_calculate_global_totals(df):

    technologies = ['Gas Turbine', 'Steam Turbine', 'Combined Cycle', 'Internal Combustion Combined Cycle', 'Integrated Solar Combined Cycle', 'Allam-Fetvedt Cycle', 'Internal Combustion', 'Unknown']

    gogpt_age_all = df.groupby('Decade')[technologies].sum().reset_index()
    gogpt_age_all.insert(0, 'Country', 'all')

    df = pd.concat([gogpt_age_all, df], sort=False)

    return df

def age_fill_in_missing_decades(df):
    decade_list = [
        '0-9 years',
        '10-19 years',
        '20-29 years',
        '30-39 years',
        '40-49 years',
        '50+ years',
    ]
    for country_sel in gogpt_country_list:
        for decade_sel in decade_list:
            # select where the country and decade match within the existing df
            df_sel = df[(df['Country']==country_sel) & (df['Decade']==decade_sel)]
            if len(df_sel) == 0:
                print(f'length is zero for this decade and country: {country_sel} {decade_sel}')
                new_df = pd.DataFrame.from_dict({
                        'Country': country_sel,
                        'Decade': decade_sel,
                        'Gas Turbine': [float(0)],
                        'Steam Turbine': [float(0)],
                        'Combined Cycle': [float(0)],
                        'Internal Combustion Combined Cycle': [float(0)],
                        'Integrated Solar Combined Cycle': [float(0)],
                        # 'Ultra-Integrated Solar Combined Cycle': [float(0)],
                        'Allam-Fetvedt Cycle': [float(0)],
                        'Internal Combustion': [float(0)],
                        'Unknown': [float(0)],
                    }, orient='columns')
                df = pd.concat([df, new_df], sort=False)

            elif len(df_sel) == 1:
                pass

            elif len(df_sel) > 1:
                print("Error!")
            else:
                print("Error! (of another kind)")

    return df


gogpt_age = age_read_and_clean(gogpt_age)
gogpt_age = age_condense_data(gogpt_age)
gogpt_age = age_calculate_global_totals(gogpt_age)
gogpt_age = age_fill_in_missing_decades(gogpt_age)

10386
this is df after groupby:              Country       Decade           Technology  Capacity (MW)
0        Afghanistan    0-9 years  Internal Combustion           59.0
1            Albania    0-9 years  Internal Combustion          110.2
2            Albania  10-19 years       Combined Cycle           97.0
3            Algeria    0-9 years       Combined Cycle        10986.0
4            Algeria    0-9 years          Gas Turbine         4336.5
...              ...          ...                  ...            ...
1019  Western Sahara    0-9 years  Internal Combustion           72.0
1020           Yemen    0-9 years          Gas Turbine          626.0
1021           Yemen  10-19 years          Gas Turbine          342.0
1022           Yemen  10-19 years        Steam Turbine           60.0
1023        Zimbabwe    0-9 years              Unknown          400.0

[1024 rows x 4 columns]
516
length is zero for this decade and country: Afghanistan 10-19 years
length is zero for this decade 

/var/folders/70/9xc3s63n0j9crf3st7fc61cm0000gn/T/ipykernel_63111/2962616733.py:43: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0-9 years' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[row, 'Decade'] = '0-9 years'


length is zero for this decade and country: India 40-49 years
length is zero for this decade and country: India 50+ years
length is zero for this decade and country: Indonesia 50+ years
length is zero for this decade and country: Iraq 30-39 years
length is zero for this decade and country: Ireland 30-39 years
length is zero for this decade and country: Ireland 50+ years
length is zero for this decade and country: Isle of Man 0-9 years
length is zero for this decade and country: Isle of Man 10-19 years
length is zero for this decade and country: Isle of Man 30-39 years
length is zero for this decade and country: Isle of Man 40-49 years
length is zero for this decade and country: Isle of Man 50+ years
length is zero for this decade and country: Israel 50+ years
length is zero for this decade and country: Jamaica 30-39 years
length is zero for this decade and country: Jamaica 40-49 years
length is zero for this decade and country: Jamaica 50+ years
length is zero for this decade and count

In [419]:
gogpt_age.tail()


,Country,Decade,Gas Turbine,Steam Turbine,Combined Cycle,Internal Combustion Combined Cycle,Integrated Solar Combined Cycle,Allam-Fetvedt Cycle,Internal Combustion,Unknown
0,Zimbabwe,10-19 years,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Zimbabwe,20-29 years,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Zimbabwe,30-39 years,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Zimbabwe,40-49 years,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,Zimbabwe,50+ years,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Additions 
- data for bar chart additions (lower-right)
- sheet '2000-2022' (for example)
- need country year capacity status

In [420]:
# create add df 
gogpt_add = gogpt_orig[['Country', 'Capacity (MW)', 'Start year', 'Status']].copy()
print(gogpt_add.shape)
gogpt_add = gogpt_add.dropna(subset=['Start year'])
gogpt_add = gogpt_add.drop(gogpt_add[gogpt_add['Start year'] == 'not found'].index) 
print(gogpt_add.shape)



(11774, 4)
(10386, 4)


In [421]:
# clean for not found

gogpt_add['Capacity (MW)'] = gogpt_add['Capacity (MW)'].fillna(1).replace('not found', 1)
# drop out rows for not found start year 
# using current year instead of 0 like for gogpt_age because need it for adding in all variable
gogpt_add['Start year'] = gogpt_add['Start year'].fillna(2023).replace('not found', 2023)

for row in gogpt_add.index:
    year = gogpt_add.at[row, 'Start year']
    if '-' in str(year):
        first_yr = int(year.split('-')[0].strip())
        gogpt_add.at[row, 'Start year'] = first_yr
    elif ',' in str(year):
        first_yr = int(year.split(',')[0].strip())
        gogpt_add.at[row, 'Start year'] = first_yr
    elif ';'in str(year):
        first_yr = int(year.split(';')[0].strip())
        gogpt_add.at[row, 'Start year'] = first_yr


#replace 'before 1992' with 1992
gogpt_add = gogpt_add.drop(gogpt_add[gogpt_add['Start year'] == 'before 1992'].index)

# remove any years before 2000
gogpt_add = gogpt_add[gogpt_add['Start year'] >= 2000]



In [422]:
gogpt_add.tail()


,Country,Capacity (MW),Start year,Status
11782,New Zealand,25.8,2004,operating
11783,New Zealand,25.8,2004,operating
11784,New Zealand,25.8,2004,operating
11785,New Zealand,25.8,2004,operating
11786,New Zealand,25.8,2004,operating


In [423]:
# check that it did it's thing
for value in gogpt_add['Start year']:
    if '-' in str(value):
        print(value)

In [424]:
# here we just want to drop any rows that don't have status == operating
gogpt_add = gogpt_add[gogpt_add['Status']=='operating']



In [425]:
# add a step where we add capacities together for a unique country year combo
gogpt_add = gogpt_add.groupby(['Country','Start year', 'Status']).sum().reset_index()


In [426]:
def add_unstack(df):
    df = df.set_index(['Country', 'Start year', 'Status'])
    # rearranges / pivots data of df
    # df = df.unstack(-1)
    df = df.unstack()

    # removes first row since: 1 or ‘columns’: remove level(s) in row.
    df = df.droplevel(0, axis=1)
    print(df.head())
    df = df.reset_index()
    df = df.rename(columns={
        'operating': 'Added (MW)', 
        'Start year': 'Year'
    })
        
    return df

In [427]:
# add missing countries
def add_missing_countries(df):
    # add any missing countries, to make sure that all countries in gcpt_country_list are in gcpt_map
    # missing_countries = [x for x in gcpt_country_list if x not in gcpt_add['Country'].tolist()]
    missing_countries = [x for x in gogpt_country_list if x not in gogpt_add['Country'].tolist()]

    print(f"Show any countries missing (which will be added below): {missing_countries}")
    
    for year in range(int(gogpt_add['Year'].min()), 
        int(max_year)+1):

        new_df = pd.DataFrame.from_dict({
                'Country': missing_countries,
                'Year': year,
                'Added (MW)': [float(0)]*len(missing_countries),
                # 'Status': 'operating',
            }, orient='columns')
        print(f'this is new df: {new_df}')
        df = pd.concat([df, new_df], sort=False)
    return df


In [428]:

def fill_in_missing_years(df):
    # remove all 
    # gogpt_country_list = 
    for country_sel in gogpt_country_list:
        for year in range(int(gogpt_add['Year'].min()), 
            int(max_year)+1):

            df_sel = df[(df['Country']==country_sel) & (df['Year']==year)]
            if len(df_sel) == 0:
                new_df = pd.DataFrame.from_dict({
                        'Country': country_sel,
                        'Year': year,
                        'Added (MW)': [float(0)],
                    }, orient='columns')
                df = pd.concat([df, new_df], sort=False)
    return df

In [429]:
# then groupby capacities by year and country
def add_calculate_global_totals(df):
    gogpt_add_all = df.groupby(['Year'])[['Added (MW)']].sum().reset_index()
    gogpt_add_all.insert(0, 'Country', 'all')
    df = pd.concat([gogpt_add_all, df], sort=False)
    
    return df

In [430]:
gogpt_add = add_unstack(gogpt_add)
# gogpt_add = add_missing_countries(gogpt_add)
gogpt_add = add_calculate_global_totals(gogpt_add)
gogpt_add = fill_in_missing_years(gogpt_add)


Status              operating
Country Start year           
Algeria 2000            123.0
        2001            300.0
        2002            418.0
        2003            123.0
        2004            292.0


In [431]:
gogpt_add.tail()

,Country,Year,Added (MW)
0,Zimbabwe,2019,0.0
0,Zimbabwe,2020,0.0
0,Zimbabwe,2021,0.0
0,Zimbabwe,2022,0.0
0,Zimbabwe,2023,0.0


## Save all to files

In [432]:
# export to various sheets of one Excel file by creating ExcelWriter object
template_name = gogpt_dash_file.split('.xlsx')[0]
save_timestamp = time.strftime('%Y-%m-%d_%H%M', time.localtime())
file_name = f'{template_name} - processed for Dash {save_timestamp}.xlsx'
with pd.ExcelWriter(gogpt_dash_path + file_name) as writer:
    gogpt_map.to_excel(writer, sheet_name='map', index=False)  
    gogpt_status.to_excel(writer, sheet_name='status', index=False)
    gogpt_age.to_excel(writer, sheet_name='age', index=False)
    gogpt_add.to_excel(writer, sheet_name='additions', index=False)
    print(f"Saved to file: {file_name}")

Saved to file: Global Oil and Gas Plant Tracker (GOGPT) compiled 2023-08-18 - processed for Dash 2023-09-21_1533.xlsx
